In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import sys
sys.path.append('../')
from ptflops import get_model_complexity_info
import json

import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

from main.efficientnet import effnetv2_s, EffNetV2_FC, EffNetV2_FC_Branch
from main.trainer_domainnet import DomainNetTrainer
from main.layout import Layout
from main.algorithms import enum_layout_wo_rdt, init_S, coarse_to_fined
from data.domainnet_dataloader import DomainNet, MultiDomainSampler

In [2]:
projectroot = '/work/lijunzhang_umass_edu/data/multibranch/'
data = 'DomainNet'
ckptroot = os.path.join(projectroot, 'checkpoint/', data, 'init/')
pretrained = 'efficientnet_v2_s-dd5fe13b.pth'

In [3]:
state_dict = torch.load(ckptroot+pretrained)
model = effnetv2_s()
eff_state_dict = model.state_dict()

In [4]:
tasks = ['real','real']
branch = 42
cls_num = {task: 345 for task in tasks}
my_model = EffNetV2_FC(tasks=tasks, branch=branch, cls_num=cls_num)

Construct EffNetV2 from Layout:
[[{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}], [{0, 1}]]


In [6]:
eff_state_dict_mtl = my_model.backbone.state_dict()

In [9]:
for pre_key, eff_key_mtl in zip(state_dict,eff_state_dict_mtl):
    print(pre_key, eff_key_mtl)
    # print(state_dict[pre_key].shape, eff_state_dict_mtl[eff_key_mtl].shape)
    print(torch.equal(state_dict[pre_key], eff_state_dict_mtl[eff_key_mtl]))

features.0.0.weight mtl_blocks.0.block.0.weight
True
features.0.1.weight mtl_blocks.0.block.1.weight
True
features.0.1.bias mtl_blocks.0.block.1.bias
True
features.0.1.running_mean mtl_blocks.0.block.1.running_mean
True
features.0.1.running_var mtl_blocks.0.block.1.running_var
True
features.0.1.num_batches_tracked mtl_blocks.0.block.1.num_batches_tracked
True
features.1.0.block.0.0.weight mtl_blocks.1.block.conv.0.weight
True
features.1.0.block.0.1.weight mtl_blocks.1.block.conv.1.weight
True
features.1.0.block.0.1.bias mtl_blocks.1.block.conv.1.bias
True
features.1.0.block.0.1.running_mean mtl_blocks.1.block.conv.1.running_mean
True
features.1.0.block.0.1.running_var mtl_blocks.1.block.conv.1.running_var
True
features.1.0.block.0.1.num_batches_tracked mtl_blocks.1.block.conv.1.num_batches_tracked
True
features.1.1.block.0.0.weight mtl_blocks.2.block.conv.0.weight
True
features.1.1.block.0.1.weight mtl_blocks.2.block.conv.1.weight
True
features.1.1.block.0.1.bias mtl_blocks.2.block.con